https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

In [341]:
import numpy
import scipy.io.wavfile
from scipy.fftpack import dct
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import os
import tensorflow as tf
import plaidml.keras

plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

<h1>1. Define Pre Processing Function</h1>

This function will apply all of the necessary pre-processsing to the speech signal. It assumes that we've already cut the speech signal down to 3 seconds. Then:

- Apply Pre-emphasis
- Framing
- Framing
- Hamming Window
- Fourier Spectrum
- Log-Mel Filter Banks
- MFCC scale
- Normalization

The function will happily accept larger segments but it will still spit out the same length vector which will confuse the network.

I think a lot of this can be done with Tensorflow function which are probably a bit faster since just the pre-processing takes some time.

In [342]:
def apply_preprocessing(signal):

    pre_emphasis = 0.97
    emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

    frame_size = .025
    frame_stride = 0.01

    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

    pad_signal_length = num_frames * frame_step + frame_length
    z = numpy.zeros((pad_signal_length - signal_length))
    pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

    indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(numpy.int32, copy=False)]
    frames *= numpy.hamming(frame_length)
    # frames *= 0.54 - 0.46 * numpy.cos((2 * numpy.pi * n) / (frame_length - 1))  # Explicit Implementation **
    NFFT = 512

    mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

    nfilt = 40

    low_freq_mel = 0
    high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
    mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
    bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = numpy.dot(pow_frames, fbank.T)
    filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * numpy.log10(filter_banks)  # dB

    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')

    mfcc -= (numpy.mean(mfcc, axis=0) + 1e-8)
    
    return mfcc

<h1>2. Importing and Preprocessing Some Data</h1>

I decided to go with 20 speakers to start off with. We are taking their names, locating all the files under their names and applying the 3-second windows to each file so we get out a list of tensors and speaker ID's. Our end result will be a TF dataset that we can directly apply to train our network.

Unfortunately, the real dataset is around 36GB, which will be too big to fit in RAM. We'll need some way to break it up in order to use it to train our network.

In [343]:
meta = pd.read_csv("../Datasets/vox1_meta.csv", delimiter = '\t', index_col = "VoxCeleb1 ID")

In [344]:
dev_loc = "../../Dataset/vox1_dev_wav/vox1_dev_wav/wav"
test_loc = "../../Dataset/wav"

train_x = []
train_y = []

counter = 0

for index in meta.iloc[0:100].index:
    dev_path = os.path.join(dev_loc, index)
    test_path = os.path.join(test_loc, index)
    if os.path.exists(dev_path):
        paths = os.walk(dev_path)
    if os.path.exists(test_path):
        paths = os.walk(test_path)
    if paths != None:
        for sub_paths in paths:
            for file in sub_paths[2]:
                sample_rate, signal = scipy.io.wavfile.read(os.path.join(sub_paths[0],file))
                signals = list(zip(*[iter(signal)]*(3*sample_rate)))
                train_x.extend([numpy.swapaxes(apply_preprocessing(numpy.array(signal)),0,1) for signal in signals])
                train_y.extend([index]*len(signals))
                counter += 1
                if(counter%100 == 0):
                    print(str(counter) + " records processed so far...")
    paths = None
    

100 records processed so far...
200 records processed so far...
300 records processed so far...
400 records processed so far...
500 records processed so far...
600 records processed so far...
700 records processed so far...
800 records processed so far...
900 records processed so far...
1000 records processed so far...
1100 records processed so far...
1200 records processed so far...
1300 records processed so far...
1400 records processed so far...
1500 records processed so far...
1600 records processed so far...
1700 records processed so far...
1800 records processed so far...
1900 records processed so far...
2000 records processed so far...
2100 records processed so far...
2200 records processed so far...
2300 records processed so far...
2400 records processed so far...
2500 records processed so far...
2600 records processed so far...
2700 records processed so far...
2800 records processed so far...
2900 records processed so far...
3000 records processed so far...
3100 records proces

In [345]:
train_x = list(map(lambda x : x.reshape((298,40,1)), train_x))

In [346]:
train_y_ = OneHotEncoder(sparse=False).fit_transform(numpy.array(train_y).reshape(-1,1))
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y_)).batch(64)

<h1>3. Building the Model</h1>

In [348]:
VGG19 = tf.keras.applications.VGG19(include_top=False, weights=None, input_shape=(298,40,1))

In [369]:
class SelfAttention(tf.keras.layers.Layer):

  #nh - output dimension of last network step
  #nk - number of attention hops
  #nc - hidden size of intermediate layer
  #T  - the sequence length, since I'm sorta used to image processing that's sort of misleading, the sequence length is simply
    #the height of the input image.
    def __init__(self, nk=4, nc=64, initializer=tf.keras.initializers.GlorotNormal()):
        super(SelfAttention, self).__init__()
        self.nh = None
        self.T = None
        self.nk = nk
        self.nc = nc
        self.initializer = initializer

  #T - number of frames we will run through at one time
    def build(self, input_shape):
    #I'm expecting a tensor in this format(batch_size, height, sequence_len, num_filters)
    #out of which I want to extract a flattened tensor height x num_filters
    #sequence len is T
        self.nh = input_shape[2] * input_shape[3]
        self.W1 = self.add_weight(
            shape=(self.nh,self.nc),
            initializer= self.initializer,
            trainable=True
        )
        self.W2 = self.add_weight(
            shape=(self.nc, self.nk),
            initializer= self.initializer,
            trainable=True
        )

  #H - Output of the VGG or other processing network
    @tf.function
    def call(self, H):
        @tf.function
        def operations(H):
            H = tf.reshape(H,tf.convert_to_tensor([H.shape[0],H.shape[1]*H.shape[2]]))
            A = tf.nn.softmax(tf.nn.tanh(tf.linalg.matmul(tf.linalg.matmul(H,self.W1),self.W2)))
            return tf.linalg.matmul(H, A, transpose_a = True)
        return tf.map_fn(operations, H)
    

In [370]:
model = tf.keras.Sequential(VGG19.layers[0:8])
[model.add(layer) for layer in VGG19.layers[11:13]]
model.add(tf.keras.layers.MaxPool2D(padding='same'))
model.add(SelfAttention())
model.add(tf.keras.layers.AveragePooling1D(pool_size=2, padding='same'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [371]:
history = model.fit(dataset, epochs=60)

Epoch 1/60
413/413 [==============================] - 8086s 20s/step - loss: 4.9907 - accuracy: 0.0664
Epoch 2/60
413/413 [==============================] - 8641s 21s/step - loss: 8.1555 - accuracy: 0.0951
Epoch 3/60
413/413 [==============================] - 7757s 19s/step - loss: 3.8140 - accuracy: 0.0945
Epoch 4/60
413/413 [==============================] - 7752s 19s/step - loss: 4.3917 - accuracy: 0.0969
Epoch 5/60
413/413 [==============================] - 7673s 19s/step - loss: 4.6378 - accuracy: 0.0384
Epoch 6/60
413/413 [==============================] - 7662s 19s/step - loss: 4.4632 - accuracy: 0.0042
Epoch 7/60
186/413 [============>.................] - ETA: 1:11:30 - loss: 4.6759 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [376]:
model.summary()

Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 298, 40, 64)       640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 298, 40, 64)       36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 20, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 20, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 20, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 10, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 74, 10, 256)     